In [1]:
# explicit import are preferred
import bokeh
import bokeh.io
import bokeh.plotting
import pandas as pd
import numpy as np

# hide warnings
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

In [2]:
# Set to output the plot in the notebook
bokeh.io.output_notebook()

Loading BokehJS ...

## Get the data

In [3]:
data_url = "\
https://raw.githubusercontent.com/WillKoehrsen/\
Bokeh-Python-Visualization/master/application/data/flights.csv"

In [4]:
flights = pd.read_csv(data_url, index_col=0)


### Build histogram


In [5]:
bins, edges = np.histogram(
    flights["arr_delay"], 
    bins=int(240 / 5), 
    range=(-120, 120),
    )

In [6]:
delays = pd.DataFrame({
    'flights': bins, 
    'left': edges[:-1], 
    'right': edges[1:],
})

In [7]:
# Create the blank plot
p = bokeh.plotting.figure(
    plot_height=400,
    plot_width=400, 
    title='Histogram of Arrival Delays',
    x_axis_label='Delay (min)', 
    y_axis_label='Number of Flights')

# Add a quad glyph
p.quad(
    bottom=0, 
    top=delays['flights'], 
    left=delays['left'],
    right=delays['right'], 
    fill_color='#262626',
    line_color='black')

# Show the plot
bokeh.io.show(p)

### Add interactivity
We basically replace the pandas dataframe with a `ColumnDataSource` object, which stores the data differently, making it accessible inside bokeh objects when entered as a `source` argument. 

In [8]:
# Convert dataframe to column data source
src = bokeh.models.ColumnDataSource(delays)
src.data.keys()

dict_keys(['right', 'left', 'flights', 'index'])

### INteractive

In [9]:
# add a column to df and update src
delays["interval"] = (
    delays['left'].astype(str) + ' to ' + delays['right'].astype(str))

# Convert dataframe to column data source
src = bokeh.models.ColumnDataSource(delays)

In [10]:
# Create the blank plot
p = bokeh.plotting.figure(
    plot_height=400,
    plot_width=400, 
    title='Histogram of Arrival Delays',
    x_axis_label='Delay (min)',
    y_axis_label='Number of Flights',
)

# Add a quad glyph with source this time
p.quad(
    bottom=0,
    top='flights',
    left='left',
    right='right',
    source=src,
    fill_color='#262626',
    line_color='#262626',
    fill_alpha=0.75,
    hover_fill_alpha=1.0,
    hover_fill_color='lightgrey',
)

# Add a hover tool referring to the formatted columns
hover = bokeh.models.HoverTool()
hover.tooltips = [
    ('Delay', '@interval'),
    ('Num of Flights', '@flights')]
# Add the hover tool to the graph
p.add_tools(hover)

# Style the plot
p.title.align = "center"
p.title.text_font_size = "14pt"

# Show the plot
bokeh.io.show(p)

### Save the plot

In [11]:
# Specify the output file and save
bokeh.io.output_file('histogram.html')